# Deep learning

# classer si les les cartons sont damaged ou non ! 

In [7]:
import pyodbc
import numpy as np
import io
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Connexion SQL Server
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-8C3VSOH;'
    'DATABASE=SuplyChain_DataWarehouse;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

# Chargement des images depuis SQL
def load_images(dataset_type):
    cursor.execute(f"SELECT image, label FROM Images WHERE dataset_type = ?", dataset_type)
    rows = cursor.fetchall() #    Stocke tous les résultats (chaque ligne = une image + un label) dans la variable rows.
    X, y = [], [] #X stockera les images (sous forme de tableaux numpy) // #y stockera les étiquettes (0 ou 1).

    for row in rows:
        try:
            img = Image.open(io.BytesIO(row[0])).convert("RGB").resize((64, 64)) #io.BytesIO(row[0]) : convertit le binaire en un flux lisible.
            X.append(np.array(img))  # Ajout de l’image au dataset
            y.append(0 if row[1] == "undamagedpackages" else 1)  # Conversion du label en chiffre
        except Exception as e:
            print(f"Erreur image : {e}")
    return np.array(X) / 255.0, np.array(y)
#np.array(X) : convertit la liste des images en tableau numpy.
#/ 255.0 : normalise les valeurs des pixels entre 0 et 1 (car ils vont de 0 à 255).
#np.array(y) : convertit la liste des étiquettes en tableau numpy.

# Chargement des données
X_train, y_train = load_images("train")
X_valid, y_valid = load_images("valid")
X_test, y_test   = load_images("test")

# Data Augmentation Quand on n’a pas beaucoup d’images dans l’ensemble d'entraînement, le modèle risque de sur-apprendre (overfitting).
# La data augmentation simule de nouvelles images à partir de celles qu’on a déjà, en appliquant des transformations aléatoires réalistes.
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)

# Architecture du modèle
model = Sequential([ #chaque couche est ajoutée l’une après l’autre (modèle linéaire)
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3), kernel_regularizer=l2(0.001)), # 1re couche convolutionnelle 
    #Conv2D(32, (3,3)) : crée 32 filtres (ou kernels) de taille 3x3.activation='relu' : applique la fonction ReLU (Rectified Linear Unit) pour introduire de la non-linéarité.
    MaxPooling2D(2, 2), #Applique une réduction de dimension 
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    MaxPooling2D(2, 2),
    Flatten(), #Transforme les données 2D en 1D (vecteur) pour pouvoir les passer à des couches entièrement connectées (Dense).
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5), #Désactive aléatoirement 50% des neurones pendant l’entraînement. #Sert à éviter que le réseau devienne dépendant de certains neurones (encore contre l’overfitting).
    Dense(1, activation='sigmoid')
])

# Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#  Calcul des poids des classes Équilibrage des classes
#Par exemple, si tu as beaucoup de undamaged et peu de damaged, cette méthode donne plus d’importance aux damaged

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

# Entraînement
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32), # applique en temps réel les transformations (rotation, zoom…)
    epochs=50, #nombre maximal d’itérations sur tout le dataset.
    validation_data=(X_valid, y_valid), #utilisé pour surveiller val_loss pendant l’entraînement.
    callbacks=[early_stop],
    class_weight=class_weights
)

# === ÉVALUATION TEST ===
y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

print("=== Résultats TEST ===")
print(classification_report(y_test, y_pred, target_names=["undamaged", "damaged"]))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["undamaged", "damaged"],
            yticklabels=["undamaged", "damaged"])
plt.xlabel("Prédiction")
plt.ylabel("Vraie classe")
plt.title("Matrice de confusion - TEST")
plt.show()

# === ÉVALUATION VALIDATION ===
y_val_pred_prob = model.predict(X_valid).flatten()
y_val_pred = (y_val_pred_prob > 0.5).astype(int)

print("=== Résultats VALIDATION ===")
print(classification_report(y_valid, y_val_pred, target_names=["undamaged", "damaged"]))

cm_val = confusion_matrix(y_valid, y_val_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens',
            xticklabels=["undamaged", "damaged"],
            yticklabels=["undamaged", "damaged"])
plt.xlabel("Prédiction")
plt.ylabel("Vraie classe")
plt.title("Matrice de confusion - VALIDATION")
plt.show()

# === COURBES D'APPRENTISSAGE ===
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Valid Acc')
plt.title('Courbe de précision')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Valid Loss')
plt.title('Courbe de perte')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'pyodbc'

In [ ]:
model.save("cnn_damage_detector.h5")


In [ ]:
# Afficher les résultats image par image
for i in range(len(y_test)):
    vrai = "damaged" if y_test[i] == 1 else "undamaged"
    predit = "damaged" if y_pred[i] == 1 else "undamaged"
    print(f"Image {i+1} : Vrai = {vrai} | Prédit = {predit}")


Image 1 : Vrai = damaged | Prédit = undamaged
Image 2 : Vrai = damaged | Prédit = damaged
Image 3 : Vrai = damaged | Prédit = damaged
Image 4 : Vrai = damaged | Prédit = undamaged
Image 5 : Vrai = damaged | Prédit = damaged
Image 6 : Vrai = damaged | Prédit = damaged
Image 7 : Vrai = damaged | Prédit = damaged
Image 8 : Vrai = damaged | Prédit = damaged
Image 9 : Vrai = damaged | Prédit = damaged
Image 10 : Vrai = damaged | Prédit = damaged
Image 11 : Vrai = damaged | Prédit = undamaged
Image 12 : Vrai = damaged | Prédit = damaged
Image 13 : Vrai = damaged | Prédit = damaged
Image 14 : Vrai = damaged | Prédit = damaged
Image 15 : Vrai = damaged | Prédit = damaged
Image 16 : Vrai = damaged | Prédit = undamaged
Image 17 : Vrai = damaged | Prédit = damaged
Image 18 : Vrai = damaged | Prédit = undamaged
Image 19 : Vrai = damaged | Prédit = damaged
Image 20 : Vrai = damaged | Prédit = damaged
Image 21 : Vrai = damaged | Prédit = damaged
Image 22 : Vrai = damaged | Prédit = damaged
Image 23 